Group 10: PERFECT TEN Presents The Effects of Facility Air Pollution and Income on Life Expectancy
Data Sources: EPA, Golden Oaks Research Group, and Global Health Data Exchange
Wisconsin: The decision to look into wisconsin is supported in the facility_df notebook

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import zipfile
# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

In [3]:
#Wisconsin Facility Data Frame
df = pd.read_csv('Facility Air Pollution Dataset - All Facilities.csv', low_memory = False)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
df_clean = df.loc[df['Rank_TRI_14'].notnull()].sort_values(by = 'Rank_TRI_14')
df_clean['Rank_TRI_14'] = list(range(15461,0, -1))
df_wi = df_clean.loc[df['State'] == 'WI']
df_wi

Unique ID                                       FacilityName  Rank_TRI_14  Rank_GHG_14   Latitude  Longitude                                   LocationAddress               City State        ZIP       County  FIPScode  PrimaryNAICS  SecondPrimaryNAICS  ThirdPrimaryNAICS                            IndustryType                          Parent_Companies_2014_GHG                     Parent_Companies_2014_TRI TRI_Air_Emissions_14_in_lbs TRI_Air_Emissions_13_in_lbs TRI_Air_Emissions_12_in_lbs TRI_Air_Emissions_11_in_lbs TRI_Air_Emissions_10_in_lbs GHG_Direct_Emissions_14_in_metric_tons GHG_Direct_Emissions_13_in_metric_tons GHG_Direct_Emissions_12_in_metric_tons GHG_Direct_Emissions_11_in_metric_tons GHG_Direct_Emissions_10_in_metric_tons     GHG_ID  Second_GHG_ID  Third_GHG_ID  Fourth_GHG_ID  Fifth_GHG_ID  Sixth_GHG_ID           TRI_ID Second_TRI_ID Third_TRI_ID Fourth_TRI_ID Fifth_TRI_ID        FRS_ID  Second_FRS_ID  Third_FRS_ID  Fourth_FRS_ID
25921     110017413547                         WISCONSIN RAPIDS PULP MILL        15366       1126.0  44.404000 -89.825500                                     950 4TH AVE N   WISCONSIN RAPIDS    WI      54495         WOOD   55141.0        322110            322121.0                NaN   Pulp and Paper,Suppliers of CO2,Waste                      NEW PAGE HOLDING CORP (100%);                              VERSO PAPER CORP                     1396152                     1693201                     1693201                     1642222                     1526981                                 387932                                 347829                                 395689                                 433395                                 372112  1006095.0            NaN           NaN            NaN           NaN           NaN  54494CNSLD950FO           NaN          NaN           NaN          NaN  110017413547            NaN           NaN            NaN
25556     110000420973                                THILMANY PAPER MILL        15311       1232.0  44.282900 -88.251800                                   600 THILMANY RD           KAUKAUNA    WI      54130    OUTAGAMIE   55087.0        322121                 NaN                NaN                    Pulp and Paper,Waste                 EXPERA SPECIALTY SOLUTIONS (100%);                    EXPERA SPECIALTY SOLUTIONS                     1033668                     1093399                      927770                     1015236                     1029652                                 339897                                 355813                                 324613                                 363798                                 385954  1002339.0            NaN           NaN            NaN           NaN           NaN  54130THLMNTHILM           NaN          NaN           NaN          NaN  110000420973            NaN           NaN            NaN
25022     110013863275               GEORGIA-PACIFIC CONSUMER PRODUCTS LP        15261        694.0  44.492500 -88.032300                                   1919 S BROADWAY          GREEN BAY    WI      54304        BROWN   55009.0        322121                 NaN                NaN                          Pulp and Paper                        KOCH INDUSTRIES INC (100%);                           KOCH INDUSTRIES INC                      788166                      817313                      749250                      873422                      903299                                 865247                                 958148                                1003065                                1021491                                1048021  1000589.0            NaN           NaN            NaN           NaN           NaN  54307FRTHW1919S           NaN          NaN           NaN          NaN  110013863275            NaN           NaN            NaN
25922     110000573692                                         BIRON MILL        15178       1124.0  44.428900 -89.781700                    

In [5]:
# ranking and setting up for weighted ranks
ranking_wi = df_wi.groupby('County').sum()
count_wi = df_wi.groupby('County').count()

weight_rank = []

for i in range(0,len(count_wi)):
    item1 = ranking_wi.iloc[i,0]
    item2 = count_wi.iloc[i,2]
    weight_rank.append(item1*item2/1000)

data = {'County': count_wi.index, 'Weighted Rank': weight_rank} 

In [7]:
# .sort_values(by = 'Weighted Rank', ascending= False)
wi_rank_df = pd.DataFrame(data)
wi_rank_df.head(5)

County  Weighted Rank
0    ADAMS          9.932
1  ASHLAND         31.988
2   BARRON        702.189
3    BROWN       8785.226
4  BUFFALO         14.545

In [9]:
#getting county sizes to review pollution per mile 
counties_size = pd.read_csv('WI counties size.csv')
for i in range(len(counties_size['Value'])):
    counties_size['Value'][i] = counties_size['Value'][i].replace(',','')

#counties_size['Value']

In [10]:
pd.to_numeric(counties_size['Value'])
counties_size['County'] = counties_size['County'].str.upper()
wi_df = counties_size.merge(right = wi_rank_df, left_on= 'County', right_on='County', how='outer')
wi_df['Weighted Rank'].fillna(0, inplace = True)
wi_df['Final Ranking'] = wi_df['Weighted Rank']*100/pd.to_numeric(wi_df['Value'])
wi_df.sort_values(by = 'Weighted Rank', ascending= False)

County    Value  Weighted Rank  Final Ranking
40    MILWAUKEE    241.4      45447.302   18826.554267
67     WAUKESHA   549.57      18118.974    3296.936514
4         BROWN   529.71       8785.226    1658.497291
12         DANE  1197.24       6366.312     531.749023
70    WINNEBAGO   434.49       3752.473     863.650026
36     MARATHON  1544.98       3624.060     234.570027
58    SHEBOYGAN   511.27       3023.166     591.305181
44    OUTAGAMIE   637.52       2865.030     449.402372
35    MANITOWOC   589.08       2409.780     409.075168
71         WOOD   793.12       1815.476     228.903066
53         ROCK   718.14       1643.404     228.841730
66   WASHINGTON    430.7       1475.248     342.523334
27    JEFFERSON   556.47       1446.945     260.022104
13        DODGE   875.63       1402.436     160.163083
51       RACINE    332.5       1195.200     359.458647
45      OZAUKEE   233.08       1099.742     471.830273
29      KENOSHA   271.99        987.107     362.920328
64     WALWORTH   555.13        816.439     147.071677
10     COLUMBIA   765.53        815.078     106.472379
31    LA CROSSE   451.69        810.981     179.543714
2        BARRON   862.71        702.189      81.393400
19  FOND DU LAC   719.55        607.170      84.381905
37    MARINETTE  1399.35        574.912      41.084218
49      PORTAGE   800.68        551.072      68.825498
68      WAUPACA   747.71        464.660      62.144414
17   EAU CLAIRE   637.98        359.392      56.332800
21        GRANT  1146.85        322.752      28.142477
55         SAUK    830.9        290.405      34.950656
22        GREEN   583.96        265.158      45.406877
7       CALUMET   318.24        221.920      69.733534
8      CHIPPEWA  1008.37        168.696      16.729573
15      DOUGLAS  1304.14        167.340      12.831444
16         DUNN   850.11        152.876      17.983085
28       JUNEAU   766.93        126.188      16.453653
50        PRICE  1254.38        100.098       7.979879
26      JACKSON   987.72         94.593       9.576904
34      LINCOLN   878.97         89.880      10.225605
56       SAWYER  1257.31         88.026       7.001137
72     ST CROIX      NaN         84.144            NaN
30     KEWAUNEE   342.52         64.635      18.870431
41       MONROE   900.78         60.858       6.756145
61  TREMPEALEAU   732.97         46.584       6.355513
42       OCONTO   997.99         45.174       4.526498
1       ASHLAND  1045.04         31.988       3.060935
62       VERNON   791.58         31.908       4.030925
57      SHAWANO   893.06         18.862       2.112064
69     WAUSHARA   626.15         17.034       2.720434
5       BUFFALO   671.64         14.545       2.165595
43       ONEIDA  1112.97         14.407       1.294464
11     CRAWFORD   570.66         13.199       2.312936
14         DOOR   481.98         12.506       2.594713
38    MARQUETTE    455.6         12.267       2.692493
0         ADAMS   645.65          9.932       1.538295
33     LANGLADE   870.64          9.375       1.076794
54         RUSK   913.59          8.500       0.930395
47       PIERCE   573.75          7.588       1.322527
48         POLK   913.96          7.219       0.789860
63        VILAS    856.6          5.061       0.590824
23   GREEN LAKE   349.44          4.699       1.344723
32    LAFAYETTE   633.59          3.479       0.549093
52     RICHLAND   586.15          3.412       0.582104
24         IOWA   762.58          2.667       0.349734
6       BURNETT   821.85          1.650       0.200767
9         CLARK  1209.82          0.000       0.000000
3      BAYFIELD  1477.86          0.000       0.000000
39    MENOMINEE   357.61          0.000       0.000000
65     WASHBURN   797.11          0.000       0.000000
60       TAYLOR   974.88          0.000       0.000000
59    ST. CROIX   722.33          0.000       0.000000
18     FLORENCE    488.2          0.000       0.000000
25         IRON   758.17          0.000       0.000000
46        PEPIN   231.98          0.000       0.0

In [11]:
wi_df.sort_values(by = 'Final Ranking', ascending= False)

County    Value  Weighted Rank  Final Ranking
40    MILWAUKEE    241.4      45447.302   18826.554267
67     WAUKESHA   549.57      18118.974    3296.936514
4         BROWN   529.71       8785.226    1658.497291
70    WINNEBAGO   434.49       3752.473     863.650026
58    SHEBOYGAN   511.27       3023.166     591.305181
12         DANE  1197.24       6366.312     531.749023
45      OZAUKEE   233.08       1099.742     471.830273
44    OUTAGAMIE   637.52       2865.030     449.402372
35    MANITOWOC   589.08       2409.780     409.075168
29      KENOSHA   271.99        987.107     362.920328
51       RACINE    332.5       1195.200     359.458647
66   WASHINGTON    430.7       1475.248     342.523334
27    JEFFERSON   556.47       1446.945     260.022104
36     MARATHON  1544.98       3624.060     234.570027
71         WOOD   793.12       1815.476     228.903066
53         ROCK   718.14       1643.404     228.841730
31    LA CROSSE   451.69        810.981     179.543714
13        DODGE   875.63       1402.436     160.163083
64     WALWORTH   555.13        816.439     147.071677
10     COLUMBIA   765.53        815.078     106.472379
19  FOND DU LAC   719.55        607.170      84.381905
2        BARRON   862.71        702.189      81.393400
7       CALUMET   318.24        221.920      69.733534
49      PORTAGE   800.68        551.072      68.825498
68      WAUPACA   747.71        464.660      62.144414
17   EAU CLAIRE   637.98        359.392      56.332800
22        GREEN   583.96        265.158      45.406877
37    MARINETTE  1399.35        574.912      41.084218
55         SAUK    830.9        290.405      34.950656
21        GRANT  1146.85        322.752      28.142477
30     KEWAUNEE   342.52         64.635      18.870431
16         DUNN   850.11        152.876      17.983085
8      CHIPPEWA  1008.37        168.696      16.729573
28       JUNEAU   766.93        126.188      16.453653
15      DOUGLAS  1304.14        167.340      12.831444
34      LINCOLN   878.97         89.880      10.225605
26      JACKSON   987.72         94.593       9.576904
50        PRICE  1254.38        100.098       7.979879
56       SAWYER  1257.31         88.026       7.001137
41       MONROE   900.78         60.858       6.756145
61  TREMPEALEAU   732.97         46.584       6.355513
42       OCONTO   997.99         45.174       4.526498
62       VERNON   791.58         31.908       4.030925
1       ASHLAND  1045.04         31.988       3.060935
69     WAUSHARA   626.15         17.034       2.720434
38    MARQUETTE    455.6         12.267       2.692493
14         DOOR   481.98         12.506       2.594713
11     CRAWFORD   570.66         13.199       2.312936
5       BUFFALO   671.64         14.545       2.165595
57      SHAWANO   893.06         18.862       2.112064
0         ADAMS   645.65          9.932       1.538295
23   GREEN LAKE   349.44          4.699       1.344723
47       PIERCE   573.75          7.588       1.322527
43       ONEIDA  1112.97         14.407       1.294464
33     LANGLADE   870.64          9.375       1.076794
54         RUSK   913.59          8.500       0.930395
48         POLK   913.96          7.219       0.789860
63        VILAS    856.6          5.061       0.590824
52     RICHLAND   586.15          3.412       0.582104
32    LAFAYETTE   633.59          3.479       0.549093
24         IOWA   762.58          2.667       0.349734
6       BURNETT   821.85          1.650       0.200767
59    ST. CROIX   722.33          0.000       0.000000
60       TAYLOR   974.88          0.000       0.000000
9         CLARK  1209.82          0.000       0.000000
25         IRON   758.17          0.000       0.000000
65     WASHBURN   797.11          0.000       0.000000
3      BAYFIELD  1477.86          0.000       0.000000
39    MENOMINEE   357.61          0.000       0.000000
20       FOREST  1014.07          0.000       0.000000
18     FLORENCE    488.2          0.000       0.000000
46        PEPIN   231.98          0.000       0.0

In [25]:
#Health Data Frame
# Read Life Expectancy File and store into Pandas data frame
healthdf = pd.read_csv("Health.csv")
healthdf.tail() #using tail becuase state summary is at the top of the file

measure_id    measure_name  location_id location_name   FIPS  sex_id   sex  age_id  age_name  year_id      metric        val      upper      lower
45985          34  Mortality risk         3638   Wood County  55141       3  Both     168  65 to 85     2010  Percentage  46.795614  48.245195  45.454433
45986          34  Mortality risk         3638   Wood County  55141       3  Both     168  65 to 85     2011  Percentage  46.442541  47.773633  45.163666
45987          34  Mortality risk         3638   Wood County  55141       3  Both     168  65 to 85     2012  Percentage  46.359673  47.703183  45.076532
45988          34  Mortality risk         3638   Wood County  55141       3  Both     168  65 to 85     2013  Percentage  46.027348  47.469959  44.661011
45989          34  Mortality risk         3638   Wood County  55141       3  Both     168  65 to 85     2014  Percentage  45.525599  46.895409  44.172333

In [13]:
#drops years that are not 2014
healthdf.drop(healthdf.index[healthdf['year_id'] != 2014], inplace = True)
#drop mortality risk
healthdf.drop(healthdf.index[healthdf['measure_name'] != 'Life expectancy'], inplace = True)
#Pivot Table of Data Frame
healthpivot = pd.pivot_table(healthdf,index=['FIPS','location_name','sex'],
                                       values=['val'],
                                       aggfunc={'val':np.mean})
#show lowest life
healthpivot = healthpivot.nsmallest(10,'val')
#rename columns
healthpivot.rename(columns={'FIPS':'FIPS','location_name':'Location','sex':'Sex','val':'Value'},
                           inplace=True)
healthpivot

Value
FIPS  location_name    sex            
55078 Menominee County Male  69.800259
                       Both  72.814850
55135 Waupaca County   Male  75.007037
55079 Milwaukee County Male  75.048078
55003 Ashland County   Male  75.221359
55001 Adams County     Male  75.224196
55113 Sawyer County    Male  75.478205
55129 Washburn County  Male  75.610243
55081 Monroe County    Male  75.680905
55057 Juneau County    Male  75.726912

In [23]:
#Income Data Frame
#Read Life Expectancy File and store into Pandas data frame
incomedf = pd.read_csv("Income.csv",encoding="ISO-8859-1")
incomedf.head(5)

id  State_Code State_Name State_ab          County            City            Place  Type Primary  Zip_Code Area_Code     ALand     AWater        Lat        Lon   Mean  Median  Stdev        sum_w
0  1011000           1    Alabama       AL   Mobile County       Chickasaw   Chickasaw city  City   place     36611       251  10894952     909156  30.771450 -88.079697  38773   30506  33101  1638.260513
1  1011010           1    Alabama       AL  Barbour County      Louisville        Clio city  City   place     36048       334  26070325      23254  31.708516 -85.611039  37725   19528  43789   258.017685
2  1011020           1    Alabama       AL   Shelby County      Columbiana  Columbiana city  City   place     35051       205  44835274     261034  33.191452 -86.615618  54606   31930  57348   926.031000
3  1011030           1    Alabama       AL   Mobile County         Satsuma      Creola city  City   place     36572       251  36878729    2374530  30.874343 -88.009442  63919   52814  47707   378.114619
4  1011040           1    Alabama       AL   Mobile County  Dauphin Island   Dauphin Island  Town   place     36528       251  16204185  413605152  30.250913 -88.171268  77948   67225  54270   282.320328

In [24]:
incomedf['County'] = incomedf['County'].str.upper()
incomedf['County'] = incomedf['County'].str.replace('COUNTY','')
#drops states that are not Wisconsin 
incomedf.drop(incomedf.index[incomedf['State_ab'] != 'WI'], inplace = True)
incomedf.head()

id  State_Code State_Name State_ab     County           City           Place     Type Primary  Zip_Code Area_Code     ALand   AWater        Lat        Lon   Mean  Median  Stdev        sum_w
31397  55011007          55  Wisconsin       WI   SHAWANO          Marion        Caroline      CDP   place     54950       715   3072038   207052  44.723328 -88.899566  61462  300000  32656    46.131494
31398  55011017          55  Wisconsin       WI   OZAUKEE       Cedarburg  Cedarburg city     City   place     53012       262  12536664   132011  43.298547 -87.989107  90405   72980  70093  2227.975295
31399  55011027          55  Wisconsin       WI   CALUMET         Chilton    Chilton city     City   place     53014       920  10984922   120045  44.028082 -88.163723  58191   49115  44167  1020.365344
31400  55011037          55  Wisconsin       WI   BUFFALO   Fountain City        Cochrane  Village   place     54629       608   2024909    54183  44.227066 -91.841054  45553   46123  31857   132.893677
31401  55011047          55  Wisconsin       WI  WALWORTH     Lake Geneva            Como      CDP   place     53147       262   9300193  1527677  42.614250 -88.505124  76360   58764  63061   507.073139

The Following Section Includes the Graphs for the presentation

In [30]:
#Bar Chart


